In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import folium

In [2]:
# Read the excel sheet and skip blank rows
raw_excel = pd.read_excel("data/DSHA LIHTC List_MAPPING.xlsx", engine='openpyxl', skiprows=[2,3], skipfooter=4, dtype=str)
# Add additional column information from the first row
raw_excel.columns = (raw_excel.columns.astype(str) + " " + raw_excel.head(1).fillna("").astype(str)).iloc[0].str.strip().values
raw_excel.rename(columns={"ALLOCATION .1 DATE": "ALLOCATION DATE", "ALLOCATION  AMOUNT": "ALLOCATION AMOUNT", "Type of Property*": "Type of Property"}, inplace=True)
raw_excel.drop(0, inplace=True)

A


In [3]:
# Find and label the rows about tax year
raw_excel["is tax"] = raw_excel["PROJECT NAME & ADDRESS"].str.contains("TAX CREDIT ALLOCATIONS")

# Function that maps each the boolean column "is tax", which is true when a row contains tax year information, to an integer equal to the tax year
def assign_to_year(x, i):
    # if the row is a tax year, increment i
    if x:
        i[0] = i[0] + 1
    # return an integer equal to the tax year
    return i[0] + 2016

# add a column for the tax year
index = [-1]
raw_excel["Tax Allocation Year"] = raw_excel["is tax"].apply(assign_to_year, args=[index])

# drop rows of tax year information and reformat
raw_excel = raw_excel.loc[~raw_excel["is tax"]].drop(columns="is tax").reset_index(drop=True)


In [4]:
# Add a column that labels the primary line for an entry
raw_excel["primary"] = ~raw_excel["County"].isna()

# Function that maps each the boolean column "primary", which is true when a row contains the primary info from the dataset, to an integer that functions as an index for primary entries
def assign_to_year(x, i):
    # if the row is primary, increment i
    if x:
        i[0] = i[0] + 1
    # return an index for the primary entries
    return i[0]

# add an index column for the primary entries
index = [-1]
raw_excel["primary"] = raw_excel["primary"].apply(assign_to_year, args=[index])

In [5]:
# Fix 'ALLOCATION AMOUNT', 'ALLOCATION DATE' swap
flipped_years = [2018, 2019, 2020, 2021, 2022]
#raw_excel["ALLOCATION AMOUNT"] = raw_excel["ALLOCATION AMOUNT"].astype(str)
#raw_excel['ALLOCATION DATE'] = raw_excel['ALLOCATION DATE'].astype(str)
tmp = raw_excel.loc[raw_excel["Tax Allocation Year"].isin(flipped_years)]['ALLOCATION AMOUNT'].copy()
tmp2 = raw_excel.loc[raw_excel["Tax Allocation Year"].isin(flipped_years)]['ALLOCATION DATE'].copy()
raw_excel.loc[raw_excel["Tax Allocation Year"].isin(flipped_years), 'ALLOCATION AMOUNT'] = tmp2.values

raw_excel.loc[raw_excel["Tax Allocation Year"].isin(flipped_years), 'ALLOCATION DATE'] = tmp.values


37     881051
38        NaN
39        NaN
40        NaN
41        NaN
        ...  
134       NaN
135       NaN
136       TBD
137       NaN
138       NaN
Name: ALLOCATION DATE, Length: 102, dtype: object


In [7]:
# Create separate dataframes for each row in a data entry
grouped_data = raw_excel.groupby("primary")
raw_data1 = grouped_data.nth(0)
raw_data2 = grouped_data.nth(1).drop(columns="Tax Allocation Year")
raw_data3 = grouped_data.nth(2).drop(columns="Tax Allocation Year")
raw_data4 = grouped_data.nth(3).drop(columns="Tax Allocation Year")
raw_data5 = grouped_data.nth(4).drop(columns="Tax Allocation Year")

# Modify the column names for each dataframe to prepare for joining
raw_data2.columns = raw_data2.columns + " 2"
raw_data3.columns = raw_data3.columns + " 3"
raw_data4.columns = raw_data4.columns + " 4"
raw_data5.columns = raw_data5.columns + " 5"

# Join the dataframes by index
flattened_data = raw_data1.join(raw_data2, how="left").join(raw_data3, how="left").join(raw_data4, how="left").join(raw_data5, how="left").dropna(axis=1, how='all').reset_index(drop=True)


In [9]:
# Convert dates back to their orginial format
flattened_data["Placed in Service Date"] = pd.to_datetime(flattened_data["Placed in Service Date"], errors='coerce').dt.strftime('%m/%d/%Y')
flattened_data["ALLOCATION DATE"] = pd.to_datetime(flattened_data["ALLOCATION DATE"], errors='coerce').dt.strftime('%m/%d/%Y')
flattened_data["Tax Credit Compliance Date"] = pd.to_datetime(flattened_data["Tax Credit Compliance Date"], errors='coerce').dt.strftime('%m/%d/%Y')
flattened_data["Extended Use Period"] = pd.to_datetime(flattened_data["Extended Use Period"], errors='coerce').dt.strftime('%m/%d/%Y')
flattened_data["Placed in Service Date 2"] = pd.to_datetime(flattened_data["Placed in Service Date 2"], errors='coerce').dt.strftime('%m/%d/%Y')



In [11]:
# Combine address fields
address_columns = ["PROJECT NAME & ADDRESS", "PROJECT NAME & ADDRESS 2", "PROJECT NAME & ADDRESS 3", "PROJECT NAME & ADDRESS 4", "PROJECT NAME & ADDRESS 5"]

# Extracts addresses from projects
def extract_address(x):
    # Project 27 has three full addresses, so we use the last one listed
    if x.name == 27:
        addr = x[address_columns].dropna().values[-1]
        return addr
    # The last two lines of the address field contain the address split between two lines, except for project 27
    else:
        addr = x[address_columns].dropna().values[-2:]
        return addr[0] + ", " + addr[1]

# Extract an address for each project
flattened_data["address"] = flattened_data.apply(extract_address, axis=1)

In [12]:
# Print the data to a csv
flattened_data.to_csv("data/processed_data.csv", index=False)

In [13]:
# Print the addresses to a seperate list
flattened_data["address"].to_csv("data/DSHA_addresses.csv", index=False)

At this point we transfer the address csv over to the geolocator to get the latitude and longitude of each project

In [14]:
# Read the file of geolocated addresses
geolocations = pd.read_csv("data/counts_per_tract.csv").drop_duplicates("input addresses")
# Join the geolocations to the flattened dataframe
geolocated_data = flattened_data.merge(geolocations, left_on="address", right_on="input addresses", how="inner")
# Remove lat,lot from unsuccessfully (not in the u.s.) geolocated address
geolocated_data.loc[geolocated_data["census tract"] == "Unable To Geolocate The Address", "lot"] = np.nan
geolocated_data.loc[geolocated_data["census tract"] == "Unable To Geolocate The Address", "lat"] = np.nan

(36,)
(42,)
(42, 25)


In [15]:
# Convert lat,lot to Shapely points
geolocated_data = gpd.GeoDataFrame(geolocated_data, geometry=gpd.points_from_xy(geolocated_data['lat'], geolocated_data['lot'], crs="EPSG:4326"))


In [16]:
# Visualize points on a map

# initialize the map and store it in a folium map object
us_map = folium.Map(location=[39.74503, -75.57203], zoom_start=14, tiles=None)

# Add background tiles
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(us_map)


# Add markers for each school
points=folium.features.GeoJson(
        geolocated_data.loc[geolocated_data["census tract"] != "Unable To Geolocate The Address"], # Full geopandas data
        control=False,
        marker = folium.CircleMarker(radius = 5, # Radius in metres
                           weight = 0, #outline weight
                           fill_color = '#d95f02', 
                           fill_opacity = 1)
        )

points.add_to(us_map)
us_map

In [17]:
# extract shape files for senate districts
senate_districts = gpd.read_file("data/Enacted-Senate-EsriShp (1).zip")

# Gets the senate district containing a point
def get_district(x):
    # Return a blank when an address could not be geolocated
    if x["census tract"] == "Unable To Geolocate The Address":
        return ""
    # Return the senate district containing the point otherwise
    else:
        return senate_districts.loc[x["geometry"].within(senate_districts["geometry"])]["DISTRICT"].values[0]

# Add a column for senate district
geolocated_data["Senate District"] = geolocated_data.apply(get_district, axis=1)

In [18]:
geolocated_data.drop(columns=["input addresses", "census tract", "lot", "lat"]).to_file("data/DSHA_districted.geojson", driver="GeoJSON")

In [19]:
geolocated_data.drop(columns=["input addresses", "census tract", "lot", "lat"]).columns

Index(['PROJECT NAME & ADDRESS', 'ALLOCATION AMOUNT', 'ALLOCATION DATE',
       'Placed in Service Date', '4% or 9% Allocation',
       'Applicable Credit Rate', 'Status Active/Non', '# of Tax Credit Units',
       'Type of Property', 'County', 'Tax Credit Compliance Date',
       'Extended Use Period', 'Tax Allocation Year',
       'PROJECT NAME & ADDRESS 2', 'ALLOCATION AMOUNT 2',
       'Placed in Service Date 2', 'Applicable Credit Rate 2',
       'PROJECT NAME & ADDRESS 3', 'PROJECT NAME & ADDRESS 4',
       'PROJECT NAME & ADDRESS 5', 'address', 'geometry', 'Senate District'],
      dtype='object')

In [25]:
geolocated_data['# of Tax Credit Units'] = geolocated_data['# of Tax Credit Units'].astype(int)
geolocated_data.groupby("Senate District").sum()["# of Tax Credit Units"].reset_index().to_csv("data/Tax_Credit_Units_per_Senate_District.csv", index=False)

/var/folders/9r/738fljg527g64ktptsnnzlpw0000gn/T/ipykernel_1341/2870917187.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  geolocated_data.groupby("Senate District").sum()["# of Tax Credit Units"].reset_index().to_csv("data/Tax_Credit_Units_per_Senate_District.csv", index=False)


In [27]:
data_noTBD = geolocated_data.loc[geolocated_data['ALLOCATION AMOUNT'] != "TBD"]
data_noTBD["ALLOCATION AMOUNT"] = data_noTBD['ALLOCATION AMOUNT'].astype(int)
data_noTBD.groupby("Senate District").sum()["ALLOCATION AMOUNT"].reset_index().to_csv("data/Allocation_Amount_per_Senate_District.csv", index=False)

/Users/emiller/opt/miniconda3/envs/DSHA/lib/python3.11/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/var/folders/9r/738fljg527g64ktptsnnzlpw0000gn/T/ipykernel_1341/714395609.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data_noTBD.groupby("Senate District").sum()["ALLOCATION AMOUNT"].reset_index().to_csv("data/Allocation_Amount_per_Senate_District.csv", index=False)
